<a href="https://colab.research.google.com/github/johnkidsm/FT-scripts/blob/main/Copy_of_Demo_Scrape_FinGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code credit: oliverwang15

https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT-v3

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install transformers==4.41.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [3]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 17.1 MB/s eta 0:00:00


In [4]:
!pip install safetensors

In [5]:
!pip install sentencepiece

In [6]:
!pip install accelerate

In [7]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.1 MB/s eta 0:00:00


In [8]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] ='0'

In [9]:
import requests
from bs4 import BeautifulSoup

def scrape_headlines(url, tag):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    elements = soup.find_all(tag, limit=30)
    headlines = [element.text.strip() for element in elements]
    return headlines

url = 'https://investorplace.com/category/market-insight/'  # Replace this with the URL of the webpage you want to scrape
headlines = scrape_headlines(url, 'h2')
print(headlines)

['Apple Intelligence: Redefining AI With the Ultimate Assistant', 'Big Tech Is Going Nuclear – Here’s the Best Way to Profit', 'Where to Find the Next Big Market Gains', 'Why You Should Be Bullish on Small-Cap Stocks Now', 'The Worst Investment Move of Their Lives', 'The Top Stock of 2024 Might Surprise You…', 'Where to Focus Your Sights as the AI Boom Rages On', 'Don’t Chase This Uranium Breakout', 'NVIDIA Is Back at Record Highs – Should You Buy?', 'When Investing in AI, Substance Matters More Now Than Ever', 'Why This Earnings Season Could Send Stocks Soaring', 'A Warning to AI Investors', 'The “Godfather” of AI’s Dire Warning… and How You Can Prepare NOW', 'Tesla’s Self-Driving Reveal Disappoints, But This Revolution Is Accelerating', 'The “Godfather” of AI’s Dire Warning… And How You Can Prepare NOW', 'Bitcoin Rally Hints at Genuine Crypto Market Breakout', 'Why this Small-Cap Lull Is a Gift', 'Make Sure Your Investments Have This Type of Protection', '🤖 xAI vs. Everyone: Can Elon

In [ ]:
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Load Models
base_model = "THUDM/chatglm2-6b"
peft_model = "oliverwang15/FinGPT_ChatGLM2_Sentiment_Instruction_LoRA_FT"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModel.from_pretrained(
    base_model,
    trust_remote_code=True,
    device_map = "auto",
    offload_folder = "offload_dir",
    load_in_8bit = True)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False

In [ ]:
def create_prompts(headlines):
    prompts = []
    for headline in headlines:
        prompt = f'''Instruction: What is the sentiment of this news? Please choose an answer from {{negative/neutral/positive}}
Input: {headline}
Answer: '''
        prompts.append(prompt)
    return prompts

prompts = create_prompts(headlines)
print(prompts)

['Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: Apple Intelligence: Redefining AI With the Ultimate Assistant\nAnswer: ', 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: Big Tech Is Going Nuclear – Here’s the Best Way to Profit\nAnswer: ', 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: Where to Find the Next Big Market Gains\nAnswer: ', 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: Why You Should Be Bullish on Small-Cap Stocks Now\nAnswer: ', 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput: The Worst Investment Move of Their Lives\nAnswer: ', 'Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\nInput

In [ ]:
for prompt in prompts:

    # Generate results
    tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
    res = model.generate(**tokens, max_length=512)
    res_sentences = [tokenizer.decode(i) for i in res]
    out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
    for sentiment in out_text:
       print(sentiment)

# Output:
# positive
# neutral
# negative

 positive
 neutral
 neutral
 neutral
 neutral
 neutral
 neutral
 neutral
 positive
 neutral
 positive
 positive
 neutral
 neutral
 neutral
 positive
 positive
 neutral
 positive
 positive
 neutral
